In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install flask flask-cors pyngrok pillow tensorflow

In [18]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from PIL import Image
import base64
import numpy as np
import io
import threading

In [19]:
import tensorflow as tf
from tensorflow.keras.models import load_model

MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/model/model.h5'

# Load model
print("Đang load model...")
model = load_model(MODEL_PATH)
print("Load model thành công!\n")

# Lấy input shape
input_shape = model.input_shape
print(f"Input Shape: {input_shape}")

# Lấy kích thước ảnh (bỏ qua batch size và channels)
if len(input_shape) == 4:
    IMG_HEIGHT = input_shape[1]
    IMG_WIDTH = input_shape[2]
    CHANNELS = input_shape[3]
else:
    IMG_HEIGHT = input_shape[1]
    IMG_WIDTH = input_shape[2]
    CHANNELS = 3

print(f"Kích thước ảnh: {IMG_WIDTH} x {IMG_HEIGHT}")
print(f"Số channels: {CHANNELS}")

# Lấy output shape
output_shape = model.output_shape
print(f"\nOutput Shape: {output_shape}")

# Xác định loại classification
if len(output_shape) == 2:
    num_classes = output_shape[1]
else:
    num_classes = output_shape[-1]

print(f"Số classes: {num_classes}")

if num_classes == 1:
    print("Model type: Binary Classification (Sigmoid)")
    CLASS_NAMES = ['Cat', 'Dog']  # 0 = Cat, 1 = Dog
else:
    print("Model type: Multi-class Classification (Softmax)")
    CLASS_NAMES = ['Cat', 'Dog']

print(f"Class names: {CLASS_NAMES}")

#summary
print("MODEL SUMMARY:")
model.summary()

Đang load model...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Load model thành công!

Input Shape: (None, 128, 128, 3)
Kích thước ảnh: 128 x 128
Số channels: 3

Output Shape: (None, 1)
Số classes: 1
Model type: Binary Classification (Sigmoid)
Class names: ['Cat', 'Dog']
MODEL SUMMARY:


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Conv2D-0 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2D-1 (Conv2D)               │ (None, 124, 124, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNormalization-2            │ (None, 124, 124, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPooling2D-3 (MaxPooling2D)   │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout-4 (Dropout)             │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2D-5 (Conv2D)               │ (None, 60, 60, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2D-6 (Conv2D)               │ (None, 58, 58, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNormalization-7            │ (None, 58, 58, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPooling2D-8 (MaxPooling2D)   │ (None, 29, 29, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout-9 (Dropout)             │ (None, 29, 29, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2D-10 (Conv2D)              │ (None, 27, 27, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2D-11 (Conv2D)              │ (None, 25, 25, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNormalization-12           │ (None, 25, 25, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ MaxPooling2D-13 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout-14 (Dropout)            │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten-15 (Flatten)            │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense-16 (Dense)                │ (None, 512)            │    18,874,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ BatchNormalization-17           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout-18 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense-19 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,452,163 (74.20 MB)

 Trainable params: 19,450,241 (74.20 MB)

 Non-trainable params: 1,920 (7.50 KB)

 Optimizer params: 2 (12.00 B)

In [20]:
app = Flask(__name__)
CORS(app)

def preprocess_image(image_bytes):
    # Mở ảnh
    image = Image.open(io.BytesIO(image_bytes))

    # Convert sang RGB hoặc Grayscale
    if CHANNELS == 3:
        image = image.convert('RGB')
    elif CHANNELS == 1:
        image = image.convert('L')

    # Resize theo kích thước model yêu cầu
    image = image.resize((IMG_WIDTH, IMG_HEIGHT))

    # Convert sang array
    img_array = np.array(image)

    # Normalize về 0-1
    img_array = img_array / 255.0

    # Nếu grayscale, thêm channel dimension
    if CHANNELS == 1:
        img_array = np.expand_dims(img_array, axis=-1)

    # Thêm batch dimension
    img_array = np.expand_dims(img_array, axis=0)

    return img_array

@app.route('/')
def home():
    return jsonify({
        'status': 'running',
        'message': 'Dog Cat Classifier API',
        'model_info': {
            'input_size': f'{IMG_WIDTH}x{IMG_HEIGHT}',
            'channels': CHANNELS,
            'classes': CLASS_NAMES
        }
    })

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'healthy',
        'model_loaded': True,
        'input_size': f'{IMG_WIDTH}x{IMG_HEIGHT}'
    })

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Lấy ảnh từ request
        if 'file' in request.files:
            file = request.files['file']
            image_bytes = file.read()
        elif request.json and 'image' in request.json:
            image_data = request.json['image']
            if ',' in image_data:
                image_data = image_data.split(',')[1]
            image_bytes = base64.b64decode(image_data)
        else:
            return jsonify({'error': 'No image provided'}), 400

        # Xử lý ảnh
        processed_image = preprocess_image(image_bytes)

        # Predict
        prediction = model.predict(processed_image, verbose=0)

        # Xử lý kết quả
        if num_classes == 1:
            # Binary (Sigmoid) - output là 1 số
            prob = float(prediction[0][0])
            if prob > 0.5:
                predicted_class = CLASS_NAMES[1]  # Dog
                confidence = prob
            else:
                predicted_class = CLASS_NAMES[0]  # Cat
                confidence = 1 - prob
        else:
            # Multi-class (Softmax)
            predicted_idx = np.argmax(prediction[0])
            predicted_class = CLASS_NAMES[predicted_idx]
            confidence = float(prediction[0][predicted_idx])

        return jsonify({
            'success': True,
            'prediction': predicted_class,
            'confidence': round(confidence * 100, 2),
            'raw_output': prediction[0].tolist()
        })

    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500


In [ ]:
NGROK_AUTH_TOKEN = '************************'

ngrok.set_auth_token(NGROK_AUTH_TOKEN)


In [22]:
from pyngrok import ngrok

port = 5000
public_url = ngrok.connect(port)

print("SERVER ĐANG CHẠY!")
print(f"Public URL: {public_url}")

def run():
    app.run(port=port)

threading.Thread(target=run).start()

SERVER ĐANG CHẠY!
Public URL: NgrokTunnel: "https://fidel-euchromatic-sheryl.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
